# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

623

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# OpenWeather Key
api_key = weather_api_key
units = "imperial"

query_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}&units={units}&q="

# define count of pass throughs 
count_one = 0
set_one = 1

print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=28dd5bd9fdcd0365d2a88aabf63d3125&units=imperial&q=


In [11]:
# Create lists to store data for dataframe
city_two = []
cloudinesses = []
dates = []
humidities = []
lats = []
lngs = []
max_temps = []
wind_speeds = []
countries = []

# loops for creating dataframe columns
for city in cities:
    try:
        response = requests.get(query_url + city.replace(" ","&")).json()
        cloudinesses.append(response['clouds']['all'])
        countries.append(response['sys']['country'])
        dates.append(response['dt'])
        humidities.append(response['main']['humidity'])
        lats.append(response['coord']['lat'])
        lngs.append(response['coord']['lon'])
        max_temps.append(response['main']['temp_max'])
        wind_speeds.append(response['wind']['speed'])
        if count_one > 48:
            count_one = 1
            set_one += 1
            city_two.append(city)
        else:
            count_one += 1
            city_two.append(city)
        print(f"Processing Record {count_one} of Set {set_one} | {city}")
    except Exception:
        print("City not found. Skipping...")
print("------------------------------\nData Retrieval Complete\n------------------------------")

Processing Record 25 of Set 12 | busselton
Processing Record 26 of Set 12 | gamba
Processing Record 27 of Set 12 | kavieng
Processing Record 28 of Set 12 | cozumel
Processing Record 29 of Set 12 | ushuaia
Processing Record 30 of Set 12 | san patricio
Processing Record 31 of Set 12 | sinop
Processing Record 32 of Set 12 | huntsville
Processing Record 33 of Set 12 | leh
Processing Record 34 of Set 12 | bengkulu
Processing Record 35 of Set 12 | nikolskoye
Processing Record 36 of Set 12 | tuktoyaktuk
Processing Record 37 of Set 12 | wembley
Processing Record 38 of Set 12 | nanortalik
City not found. Skipping...
Processing Record 39 of Set 12 | isangel
Processing Record 40 of Set 12 | chokurdakh
Processing Record 41 of Set 12 | port alfred
Processing Record 42 of Set 12 | doka
Processing Record 43 of Set 12 | barrow
Processing Record 44 of Set 12 | qaanaaq
Processing Record 45 of Set 12 | butaritari
Processing Record 46 of Set 12 | beringovskiy
Processing Record 47 of Set 12 | matagami
Proc

Processing Record 12 of Set 16 | alenquer
Processing Record 13 of Set 16 | qasigiannguit
Processing Record 14 of Set 16 | kamaishi
Processing Record 15 of Set 16 | isla mujeres
Processing Record 16 of Set 16 | taoudenni
Processing Record 17 of Set 16 | grand river south east
Processing Record 18 of Set 16 | houma
Processing Record 19 of Set 16 | seoul
Processing Record 20 of Set 16 | ewo
Processing Record 21 of Set 16 | sao gabriel da cachoeira
Processing Record 22 of Set 16 | springbok
Processing Record 23 of Set 16 | akureyri
Processing Record 24 of Set 16 | pacific grove
Processing Record 25 of Set 16 | dikson
Processing Record 26 of Set 16 | eirunepe
Processing Record 27 of Set 16 | stornoway
Processing Record 28 of Set 16 | komsomolskiy
Processing Record 29 of Set 16 | lhokseumawe
Processing Record 30 of Set 16 | alyangula
Processing Record 31 of Set 16 | elban
Processing Record 32 of Set 16 | wahiawa
Processing Record 33 of Set 16 | dicabisagan
Processing Record 34 of Set 16 | la

Processing Record 49 of Set 19 | mitsamiouli
Processing Record 1 of Set 20 | anshun
Processing Record 2 of Set 20 | awbari
Processing Record 3 of Set 20 | verkh-usugli
Processing Record 4 of Set 20 | huilong
Processing Record 5 of Set 20 | dalvik
Processing Record 6 of Set 20 | flinders
Processing Record 7 of Set 20 | mantua
Processing Record 8 of Set 20 | potsdam
Processing Record 9 of Set 20 | bud
Processing Record 10 of Set 20 | lyubar
Processing Record 11 of Set 20 | port-gentil
Processing Record 12 of Set 20 | ahipara
Processing Record 13 of Set 20 | coruripe
Processing Record 14 of Set 20 | kaohsiung
Processing Record 15 of Set 20 | comodoro rivadavia
Processing Record 16 of Set 20 | porvoo
City not found. Skipping...
City not found. Skipping...
Processing Record 17 of Set 20 | avera
Processing Record 18 of Set 20 | inta
Processing Record 19 of Set 20 | camacha
Processing Record 20 of Set 20 | winnemucca
Processing Record 21 of Set 20 | vesoul
Processing Record 22 of Set 20 | boe

Processing Record 33 of Set 23 | phichit
Processing Record 34 of Set 23 | kangaatsiaq
Processing Record 35 of Set 23 | penzance
Processing Record 36 of Set 23 | iquitos
Processing Record 37 of Set 23 | evensk
Processing Record 38 of Set 23 | alpena
Processing Record 39 of Set 23 | geraldton
Processing Record 40 of Set 23 | baykit
Processing Record 41 of Set 23 | manzhouli
Processing Record 42 of Set 23 | taseyevo
Processing Record 43 of Set 23 | azul
Processing Record 44 of Set 23 | reconquista
Processing Record 45 of Set 23 | guadalajara
Processing Record 46 of Set 23 | mirnyy
Processing Record 47 of Set 23 | constantine
Processing Record 48 of Set 23 | matara
------------------------------
Data Retrieval Complete
------------------------------


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [17]:
# Create dataframe
weather_dict = {
    "City": city_two,
    "Cloudiness": cloudinesses,
    "Country": countries,
    "Date": dates,
    "Humidity": humidities,
    "Lat": lats,
    "Lng": lngs,
    "Max Temp": max_temps,
    "Wind Speed": wind_speeds
}

In [18]:
weather_df = pd.DataFrame(weather_dict)

# show the top of the dataframe
weather_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,busselton,100,AU,1604186482,79,-33.65,115.33,59.00,1.97
1,gamba,89,GA,1604186482,90,-2.65,10.00,76.77,5.35
2,kavieng,2,PG,1604186482,72,-2.57,150.80,85.19,10.22
3,cozumel,75,MX,1604186483,88,20.51,-86.95,79.00,11.41
4,ushuaia,90,AR,1604186483,87,-54.80,-68.30,50.00,6.93
5,san patricio,0,ML,1604186483,17,13.30,-4.90,81.64,6.22
6,sinop,52,TR,1604186483,73,41.77,34.87,48.33,4.14
7,huntsville,1,US,1604186005,55,34.73,-86.59,66.20,8.05
8,leh,11,IN,1604186484,74,34.17,77.58,27.70,1.01
9,bengkulu,100,ID,1604186235,87,-3.80,102.27,75.38,4.00


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [21]:
#  Get the indices of cities that have humidity over 100%.
humid = weather_df.loc[weather_df["Humidity"] > 100]
humid

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression